In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from collections import Counter
import ast
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('filenames_dataset.csv', index_col=0)
df.tags_list = df.tags_list.astype(object)
df

In [ ]:
# Flatten the list of tags
all_tags = [ast.literal_eval(sublist) for sublist in df['categories'].values if pd.isna(sublist) == False]
merged_list = [item for sublist in all_tags for item in sublist]
tag_counts = Counter(merged_list)
tag_counts

In [ ]:
categories = list(tag_counts.keys())

In [ ]:
from wordcloud import WordCloud

# Generate word cloud
wordcloud = WordCloud(width=800 * 2, height=400 * 2, background_color='white').generate(
    " ".join(
        w
        for w in merged_list
        # if w.isupper()
    )
)
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()